In [236]:
import numpy as np
from pandas import read_csv, DataFrame
from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import datasets

In [237]:
breast = datasets.load_breast_cancer()
X = breast.data
Y = breast.target
#Y = np.reshape(breast_labels, (569, 1))
ncol = X.shape[1]

In [238]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=seed(2017))
X_test.shape

(285, 30)

In [239]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)

In [240]:
cm = confusion_matrix(Y_test, y_pred)
print(cm)
print('Accuracy: ' + str(accuracy_score(Y_test, y_pred)))

[[ 97   7]
 [ 21 160]]
Accuracy: 0.9017543859649123


In [241]:
input_dim = Input(shape=(ncol, ))
# DEFINE THE DIMENSION OF ENCODER
encoding_dim = 10
# DEFINE THE ENCODER LAYER
encoded = Dense(encoding_dim, activation='relu')(input_dim)
# DEFINE THE DECODER LAYER
decoded = Dense(ncol, activation='sigmoid')(encoded)
# COMBINE ENCODER AND DECODER INTO AN AUTOENCODER MODEL
autoencoder = Model(input_dim, decoded)

In [242]:
# CONFIGURE AND TRAIN THE AUTOENCODER
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(X_train, X_train, epochs=50, batch_size=100, shuffle=True, validation_data=(X_test, X_test), verbose=0)

In [243]:
encoder = Model(input_dim, encoded)
encoded_input = Input(shape=(encoding_dim, ))
encoded_out = encoder.predict(X_test)
encoded_out.shape

(285, 10)

In [244]:
X_train, X_test, Y_train, Y_test = train_test_split(encoded_out, Y_test, test_size=0.5, random_state=seed(2017))

classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)

cm = confusion_matrix(Y_test, y_pred)
print(cm)
print('Accuracy: ' + str(accuracy_score(Y_test, y_pred)))

[[46  9]
 [ 3 85]]
Accuracy: 0.916083916083916
